# The adapted v-HOTMAP algorithm (pre-alpha verison)
## Sentinel-2 satellite imagery

The developed application allows to detect hotspots for a series of volcanoes from the list: Kilauea, Nyiragongo, Nyamuragira, Erta Ale, Villarrica, Yasur, Stromboli, Etna. Further the list of objects will be enlarged. 

In [1]:
import geemap
import io
import pandas as pd
from datetime import date
from ipyfilechooser import FileChooser
import ipywidgets as widgets
import os
import ee
import requests
import numpy as np

In [2]:
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize

In [3]:
#df with predefined volcanoes and their coordinates
data_volc = {'volcano': ['Kilauea', 'Nyiragongo', 'Nyamuragira', 'Erta Ale', 'Villarrica', 'Yasur', 'Stromboli', 'Etna'],
        'x_coords': [-155.287, 29.25, 29.2, 40.67, -71.93, 169.447, 15.213, 14.999], 
        'y_coords': [19.421, -1.52, -1.408, 13.6, -39.42, -19.532, 38.789, 37.748] }
volcanoes = pd.DataFrame(data_volc)

In [4]:
Map = geemap.Map()
Map.add_basemap('TERRAIN')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Select the volcano, date interval and them click submit to show on the map.
Select the folder and click the export button to save on the computer.

In [13]:
style = {'description_width': 'initial'}

volcano_widget = widgets.Dropdown(
    description='Select Volcano:',
    options=[
        'Kilauea',
        'Nyiragongo', #add volcanoes to the list
        'Nyamuragira',
        'Erta Ale',
        'Villarrica',
        'Yasur',
        'Stromboli',
        'Etna'
    ],
    value='Kilauea',
    style=style,
)

buffer_widget = widgets.Text(
    description='Buffer distance:', value= '3000', max_height=20,  style=style
)

start_day = widgets.DatePicker(
    description='Start date: ',
    disabled=False
)
end_day = widgets.DatePicker(
    description='End date: ',
    disabled=False
)

submit = widgets.Button(
    description='Submit', button_style='primary', tooltip='Click to submit', style=style
)
clear = widgets.Button(
    description='Clear selection', tooltip='Click to delete the selection', style=style
)
       
export = widgets.Button(
    description='Export data', button_style='info', tooltip='Click to export tif', style=style
)

export_table = widgets.Button(
    description='Export table', button_style='info', tooltip='Click to export table', style=style
)

fc = FileChooser('')
fc.show_only_dirs = True

#output = widgets.Output()

full_widget = widgets.VBox(
    [
        widgets.HBox([volcano_widget, buffer_widget]),
        widgets.HBox([start_day, end_day]),
        widgets.HBox([submit, clear]),
        fc,
        widgets.HBox([export, export_table])
    ]
)

display(full_widget)

In [6]:
List = []
hotmap_list = []

visParams = {
'bands': ['B1'],
'palette': ['#FF0000']
    }

# def on_clear_clicked(b):
#     global Map
#     Map.clear_layers()
#     Map.add_basemap('TERRAIN')
    
# clear.on_click(on_clear_clicked)
    
def on_submit_clicked(b):
    
    global start 
    start = start_day.value.strftime('%Y-%m-%d')
    global finish
    finish = end_day.value.strftime('%Y-%m-%d')
    global volcano
    volcano = volcano_widget.value
    global buffer_dist
    buffer_dist = buffer_widget.value

    start = ee.Date(start)
    finish = ee.Date(finish)
    #access the volcano name and select predefined coordinates
    #volcano = volcano_widget.value #access the value from widget

    volcano_df = volcanoes[volcanoes['volcano'].str.contains(volcano)]
    x_coords = volcano_df.iat[0, 1]
    y_coords = volcano_df.iat[0, 2]
    #create buffer area around the volcano
    #buffer_dist = buffer_widget.value #access the value from widget

    point_volcano = ee.Geometry.Point([x_coords, y_coords])
    buffer = point_volcano.buffer(int(buffer_dist))
    bbox = buffer.bounds()
    global roi
    roi = bbox
    Map.addLayer(bbox, {'color': 'blue'},
                 'Area of interest', False)
    zoom = 14
    Map.setCenter(x_coords, y_coords, zoom)
    
    s2_coll = collection_retr(start, finish, roi)
    imagelist= s2_coll.toList(s2_coll.size()) #find the size of the collection
    list_length_raw = ee.String(imagelist.length())
    list_length_str = list_length_raw.getInfo()
    image_number_auto = list_length_str-1
    
    idlist = imagelist.map(addlist) # Make a list with all the image ID's (To avoid doubles) modify later, duplications still present
    
    hotmap_ser = hotmap_series(imagelist, image_number_auto)
    hotmap_collection = ee.ImageCollection.fromImages(hotmap_ser)
    Map.add_time_slider(hotmap_collection, visParams)

    
    
submit.on_click(on_submit_clicked)

In [7]:
def on_export_click(b):
    start = start_day.value.strftime('%Y-%m-%d')
    finish = end_day.value.strftime('%Y-%m-%d')
    volcano = volcano_widget.value
    buffer_dist = buffer_widget.value

    start = ee.Date(start)
    finish = ee.Date(finish)
    
    volcano_df = volcanoes[volcanoes['volcano'].str.contains(volcano)]
    x_coords = volcano_df.iat[0, 1]
    y_coords = volcano_df.iat[0, 2]

    point_volcano = ee.Geometry.Point([x_coords, y_coords])
    buffer = point_volcano.buffer(int(buffer_dist))
    bbox = buffer.bounds()
    roi = bbox
    s2_coll = collection_retr(start, finish, roi)
    imagelist= s2_coll.toList(s2_coll.size()) #find the size of the collection
    list_length_raw = ee.String(imagelist.length())
    list_length_str = list_length_raw.getInfo()
    image_number_auto = list_length_str-1
    imagelist= s2_coll.toList(s2_coll.size())
    
    directory = fc.value
    
    # hotmap_series_export(imagelist, image_number_auto, directory)
    
    for x in range(0, image_number_auto): # export each Binary image
        imageID1 = imagelist.get(x)
        image1 = ee.Image(imageID1)

        image_date2 = ee.Date(image1.get('system:time_start')).format('YYYY-MM-dd')
        image1_hotmap = HOTMAP(image1)
        
        hotmap_list.append(image1_hotmap)
        # if image_date2.getInfo() not in List:
        #     continue
        List.append(image_date2.getInfo())

        description = 'Export_S2_HOTMAP_Mask_image_' + image_date2.getInfo()
        export_name = 'S2-HOTMAP_' + volcano + '_' + image_date2.getInfo() + '.tif'

        image_to_export = image1_hotmap.toInt() #.select(['B1'])
        url = image_to_export.getDownloadURL( {
            'bands' : ['B1'],
            'name' : export_name,
            'region' : roi,
            'scale' : 20,
            'format': 'GEO_TIFF'
        })
        response = requests.get(url)

        dir_file = directory
        dir_file.replace(r'\\', '/')

        complete_name = os.path.join(dir_file, export_name)
        print(complete_name) #remove after, just for test

        with open(complete_name, 'wb') as fd:
            fd.write(response.content)
    
export.on_click(on_export_click)

In [8]:
def table_export(b):
    start = start_day.value.strftime('%Y-%m-%d')
    finish = end_day.value.strftime('%Y-%m-%d')
    volcano = volcano_widget.value
    buffer_dist = buffer_widget.value
    
    start = ee.Date(start)
    finish = ee.Date(finish)
    
    volcano_df = volcanoes[volcanoes['volcano'].str.contains(volcano)]
    x_coords = volcano_df.iat[0, 1]
    y_coords = volcano_df.iat[0, 2]
    
    point_volcano = ee.Geometry.Point([x_coords, y_coords])
    buffer = point_volcano.buffer(int(buffer_dist))
    bbox = buffer.bounds()
    roi = bbox
    
    s2_coll = collection_retr(start, finish, roi)
    imagelist= s2_coll.toList(s2_coll.size()) #find the size of the collection
    list_length_raw = ee.String(imagelist.length())
    list_length_str = list_length_raw.getInfo()
    image_number_auto = list_length_str-1
    imagelist= s2_coll.toList(s2_coll.size())
    directory = fc.value
    
    # hotmap_series_table(imagelist, image_number_auto, directory)
    
    List_values = []
    List = []
    
    for x in range(0, image_number_auto):
        imageID1 = imagelist.get(x)
        image1 = ee.Image(imageID1)

        image_date2 = ee.Date(image1.get('system:time_start')).format('YYYY-MM-dd')
        image1_hotmap = HOTMAP(image1)

        hotmap_list.append(image1_hotmap)
        # if image_date2.getInfo() in List:
        #     continue
        List.append(image_date2.getInfo())   

        image_to_export = image1_hotmap.toInt() #.select(['B1'])
        url = image_to_export.getDownloadURL( {
            'bands' : ['B1'],
            'name' : 'Hotmap',
            'region' : roi,
            'scale' : 20,
            'format': 'NPY'
        })
        response = requests.get(url)
        numpy_array_raster = np.load(io.BytesIO(response.content))
        numpy_array_raster = numpy_array_raster.tolist()
        sum_positive = 0

        for elem in numpy_array_raster:
            for el in elem:
                if el[0] == 1:
                    sum_positive += 1

        List_values.append(sum_positive)

    print(List_values)
    #if len(List_values) == len(List):
    final_stats = pd.DataFrame( {
        'Date': List,
        'Pixel_number': List_values
    })
    print(final_stats)
    dir_file = directory
    dir_file.replace(r'\\', '/')
    print(volcano)

    # if not volcano:
    #     export_name = 'S2-HOTMAP' + '.csv'
    # else:

    export_name = 'S2-HOTMAP_' + volcano + '.csv'

    complete_name = os.path.join(dir_file, export_name)
    print(complete_name)
    final_stats.to_csv(complete_name)

export_table.on_click(table_export)

In [9]:
def collection_retr(start, finish, roi):
    collection = "COPERNICUS/S2" # Select sensor
    s2_coll = (ee.ImageCollection(collection) # Select the collection of images
        .filterDate(start, finish)
        .filterBounds(roi))
    return s2_coll

In [10]:
def addlist(list):
  return ee.Image(list).id()

# idlist = imagelist.map(addlist) # Make a list with all the image ID's (To avoid doubles for Kilauea)

In [11]:
def HOTMAP(image):
  # Variable selection (2 SWIR and 1 NIR(B8A))
  alpha1 = image.expression('B12/B11', {'B12': image.select('B12'), 'B11' : image.select('B11')})
  alpha2 = image.expression('B12/B8A', {'B12': image.select('B12'), 'B8A' : image.select('B8A')})

  beta1 = image.expression('B11/B8A', {'B11': image.select('B11'), 'B8A' : image.select('B8A')})

  B12 = image.select('B12').divide(10000) # Reflectance is scaled by 10000
  B11 = image.select('B11').divide(10000)
  B8  = image.select('B8A').divide(10000)
  # Select the cloud band and rescale from 60 to 20
  Cloud = image.select('QA60')
  proj = Cloud.projection().getInfo()
  crs = proj['crs']
  Clouds = Cloud.resample().reproject(**{'crs' : crs, 'scale' : 20.0})

  #Preparing the Alpha parameter

  alpha_A_raw = image.where(alpha1.gte(1.4),1) # Make binary map for each image (gte == greater than or equal; neq == not equal)
  alpha_A = alpha_A_raw.where(alpha_A_raw.neq(1), 0) # second parameter is the new value

  alpha_B_raw = image.where(alpha2.gte(2.0),1)
  alpha_B = alpha_B_raw.where(alpha_B_raw.neq(1),0)

  alpha_C_raw = image.where(B12.gte(0.6),1)
  alpha_C = alpha_C_raw.where(alpha_C_raw.neq(1),0)


  Alpha_total = image.expression('alpha + alphaa + alphaaa', { 'alpha' : alpha_A, 'alphaa': alpha_B, 'alphaaa': alpha_C}) # sum the binary images

  Alpha_raw = Alpha_total.where(Alpha_total.eq(3),1) # only those for which all 3 are correct 
  Alpha = Alpha_raw.where(Alpha_total.neq(3),0) # Use Alpha_total here

  # preparing the Beta parameter
  s = image.where(B11.gt(1),1) # Saturation values above 1 for both bands
  ss= s.where(B12.gt(1),1)
  S= ss.where(ss.neq(1),0)

  beta_A_raw = image.where(beta1.gte(2),1)
  beta_A = beta_A_raw.where(beta_A_raw.neq(1),0)

  beta_B_raw = image.where(B11.gte(0.5),1)
  beta_B = beta_B_raw.where(beta_B_raw.neq(1),0)

  beta_AB = image.expression('beta+betaa', { 'beta': beta_A, 'betaa': beta_B,}) # sum the binary images

  beta_C_raw = beta_AB.where(beta_AB.eq(2),1)
  beta_C = beta_C_raw.where(beta_AB.neq(2),0)

  beta_D_raw = beta_C.where(beta_C.eq(1),1)
  beta_D = beta_D_raw.where(S.eq(1),1)

  Beta = beta_D.where(beta_D.neq(1),0)

  # Combining both parameters

  Hot_pixels_raw = image.where(Beta.eq(0),0)
  Hot_pixels_raw2 = Hot_pixels_raw.where(Alpha.eq(0),0)
  Hot_pixels_raw3 = Hot_pixels_raw2.where(Beta.eq(1),1)
  Hot_pixels_raw4 = Hot_pixels_raw3.where(Alpha.eq(1),1)
  Hot_pixels = Hot_pixels_raw4.where(Clouds.eq(1<<10),0) # Setting all rescaled cloud pixels (value of 1 in the 10th bit) to zero

  # First clustering
  
  Cluster = Hot_pixels.connectedComponents(connectedness = ee.Kernel.square(1), maxSize = 128) # Find all the clusters in the image (square(1) == 8 pixel search area)
  Cluster = Cluster.select(['labels']) # Each cluster gets a unique label (int)
  Alpha = Alpha.addBands(Cluster.select(['labels'])) # for each Alpha pixel, the corresponding label is added (0 value (beta) pixels also included)
  True_clusters = Alpha.reduceConnectedComponents(reducer = ee.Reducer.mean(), labelBand = 'labels') # The mean Alpha value of each cluster is calculated, if 1 alpha is present => retain cluster
  Hotspot = Hot_pixels.where(True_clusters.gt(0),1) # Every cluster with mean Alpha value above 0 is retained
  Hotspot = Hotspot.where(True_clusters.eq(0),0)
  
  # Cloud clustering
  Cloud_seethrough_raw = image.where(Clouds.eq(1<<10),1) # Cloud images are included
  Cloud_seethrough = Cloud_seethrough_raw.where(Cloud_seethrough_raw.neq(1),0)

  # new Alpha parameters
  gamma_A_raw = image.where(B12.gte(0.9),1)
  gamma_A = gamma_A_raw.where(gamma_A_raw.neq(1),0)

  gamma_B_raw = image.where(alpha1.gte(1.4),1)
  gamma_B = gamma_B_raw.where(gamma_B_raw.neq(1),0)

  gamma_C_raw = image.where(alpha2.gte(1.65),1)
  gamma_C = gamma_C_raw.where(gamma_C_raw.neq(1),0)

  
  gamma_ABC = image.expression('gamma + gammaa + gammaaa + gammaaaa', { 'gamma': Cloud_seethrough, 'gammaa': gamma_A, 'gammaaa' : gamma_B, 'gammaaaa':gamma_C})

  gamma_raw = image.where(gamma_ABC.eq(4),1)
  gamma = gamma_raw.where(gamma_ABC.neq(4),0)
  H= image.where(Beta.eq(0),0)
  H2 = H.where(gamma.eq(0),0)
  # Original Beta is included to prevent loss of clear image Beta-pixels
  H3 = H2.where(Beta.eq(1),1)
  H4 = H3.where(gamma.eq(1),1)
  # Second clustering
  Cluster = H4.connectedComponents(connectedness = ee.Kernel.square(1), maxSize = 128) # Similar approach
  Cluster = Cluster.select(['labels'])
  gamma = gamma.addBands(Cluster.select(['labels']))
  True_clusters = gamma.reduceConnectedComponents(reducer = ee.Reducer.mean(), labelBand = 'labels')
  Hotspot2 = H4.where(True_clusters.gt(0),1)
  Hotspot2 = Hotspot2.where(True_clusters.eq(0),0)

  final = Hotspot.where(Hotspot2.gt(0),1)
  clipped = final.clip(roi)
  Mask = clipped.selfMask()

  return ee.Image(Mask) 

In [12]:
List = []
hotmap_list = []
#add here the solution to sum up all the pixels in the raster

#try idlist
def hotmap_series(imagelist, image_number_auto):
    for x in range(0, image_number_auto): # export each Binary image to google drive 
        imageID1 = imagelist.get(x)
        image1 = ee.Image(imageID1)

        image_date2 = ee.Date(image1.get('system:time_start')).format('YYYY-MM-dd')
        image1_hotmap = HOTMAP(image1)
        
        hotmap_list.append(image1_hotmap)
        if image_date2.getInfo() in List:
            continue
        List.append(image_date2.getInfo())
        Map.addLayer(image1_hotmap, visParams, "Date: " + image_date2.getInfo()) #add separate layers
        
    return hotmap_list     